# A/B - Testing
 
Cette présentation est inspirée du cours de Udacity sur l'[A/B testing](https://www.udacity.com/course/ab-testing--ud257)

Un A/B testing est un test qui vise comparer deux versions d'un même objet dans le but de déterminer la plus efficace.
Par exemple, il permet aux marketeurs de déterminer quelle version d’une page web est celle qui convertit le plus de visiteurs en acheteurs.

L'A/B testing permet de savoir si les différences de résultats sont dues aux modifications apportées et non au hasard. C'est un outil d'aide à la décision.


## Données

Les individus du test doivent être aléatoirement en deux groupes :

* Groupe de contrôle (A) : groupe d'individus qui utilisent la version actuelle
* Groupe de traitement (B) : groupe d'individus qui utilisent la nouvelle version

## Test d'égalité de proportions

Ce rappel sur le test d'égalité de proportion permet de mieux comprendre les calculs effectués au cours d'un AB-test. On utilise les notations suiventes : 

* N : nombre total d'individu
* N<sub>A</sub> : le nombre du groupe de contrôle
* N<sub>B</sub> : le nombre du groupe de traitement
* p<sub>A</sub> : le taux de conversion du groupe de contrôle 
* p<sub>B</sub> : le taux de conversion du groupe de traitement
* P<sub>A</sub> : l'estimateur du taux de conversion du groupe de contrôle 
* P<sub>B</sub> : l'estimateur du taux  de conversion du groupe de traitement
* d = p<sub>B</sub> - p<sub>A</sub> 

Les hypothèses à tester sont :

* H<sub>0</sub> : p<sub>B</sub> - p<sub>A</sub> = 0 ou p<sub>B</sub> = p<sub>A</sub>
* H<sub>1</sub> : p<sub>B</sub> - p<sub>A</sub> != 0

L'hypothèse nulle permet de conclure :
* La différence est significativement positive
* La différence est significativement négative
* Il n'y a pas de différence significative

La différence entre P<sub>A</sub> et P<sub>B</sub> suit asymptotiquement une loi normale d'espèrance d. Sous l'H<sub>0</sub> d vaut 0, en divisant cette différence par l'écart-type on obtient une loi normale centrée réduite.

<img src="images/stat_test.png">

La zone de rejet (|U| ≥ u<sub>1-α/2</sub>) avec un seuil `α` est définie ainsi:
<img src="images/intervalle_confiance.png">

**Sources :**

[Cours A/B Testing sur Udacity](https://classroom.udacity.com/courses/ud257)  
[Tests de comparaisons proportions, Yohann Foucher](http://www.divat.fr/images/Biostats/Teaching/DCEO2_Cours6_Comparaisons_Proportions.pdf)

## Applications

In [1]:
import math
import pandas as pd
import seaborn as sns
import statsmodels.stats.api as sms
import scipy.stats as ss 
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("data/ab_data.csv")
data.columns = data.columns.str.upper()
data.shape

(294478, 5)

In [3]:
data.head()

,USER_ID,TIMESTAMP,GROUP,LANDING_PAGE,CONVERTED
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [4]:
 data['USER_ID'][ data['CONVERTED'] == 1].nunique() /data['USER_ID'].nunique()

0.12104245244060237

In [5]:
data.isnull().sum()

USER_ID         0
TIMESTAMP       0
GROUP           0
LANDING_PAGE    0
CONVERTED       0
dtype: int64

In [6]:
data['GROUP'].value_counts(normalize=True)

treatment    0.500126
control      0.499874
Name: GROUP, dtype: float64

In [7]:
data['LANDING_PAGE'].value_counts(normalize=True)

old_page    0.5
new_page    0.5
Name: LANDING_PAGE, dtype: float64

Il y a un problème de données : les pourcentages ne correspondent pas:
* Certains individus du groupe de traitement n'ont pas utilisés la nouvelle page

In [8]:
# number of times the new_page in the landing_page and the treatment in the group do not line up
data[((data['GROUP'] == 'treatment') == (data['LANDING_PAGE'] == 'new_page')) == False].shape[0]

3893

Suppression des données incorrectes

In [9]:
data = data.drop(data[((data['GROUP'] == 'treatment') == (data['LANDING_PAGE'] == 'new_page')) == False].index)
data = data.drop_duplicates(subset=['USER_ID'])
data.shape

(290584, 5)

In [10]:
data['GROUP'].value_counts(normalize=True)

treatment    0.500062
control      0.499938
Name: GROUP, dtype: float64

In [11]:
data['LANDING_PAGE'].value_counts(normalize=True)

new_page    0.500062
old_page    0.499938
Name: LANDING_PAGE, dtype: float64

In [12]:
data['CONVERTED'].mean()

0.11959708724499628

In [13]:
data.groupby(['LANDING_PAGE'])['CONVERTED'].mean()

LANDING_PAGE
new_page    0.118808
old_page    0.120386
Name: CONVERTED, dtype: float64

## A/B Testing

In [14]:
data_g = data.groupby(['LANDING_PAGE'], as_index=True).agg(
              {'USER_ID': 'count', 'CONVERTED' : sum}).rename(
         columns={'USER_ID': 'VISITEURS', 'CONVERTED': 'CONVERSIONS'},
         index={'new_page': 'traitement', 'old_page': 'controle'}).rename_axis(None, axis=0)
data_g['PROBA'] = data_g['CONVERSIONS'] / data_g['VISITEURS']

proba_globale = data_g['CONVERSIONS'].sum() / data_g['VISITEURS'].sum()
proba_controle = data_g['PROBA']['controle']
proba_traitement = data_g['PROBA']['traitement']
nb_visiteurs_controle = data_g['VISITEURS']['controle']
nb_visiteurs_traitement = data_g['VISITEURS']['controle']

print('Probabilité groupe de controle: {:.4f}'.format(proba_controle))
print('Probabilité groupe de traitement: {:.4f}'.format(proba_traitement))
print('Probabilité globale: {:.4f}'.format(proba_globale))

Probabilité groupe de controle: 0.1204
Probabilité groupe de traitement: 0.1188
Probabilité globale: 0.1196


**Réalisons un test d'égalité des proportions**

In [15]:
alpha = 0.05
std = math.sqrt(proba_globale * (1 - proba_globale) * (1 / nb_visiteurs_controle + 1 / nb_visiteurs_traitement))
u_alpha = ss.norm.ppf(1 - alpha / 2) #  quantile d’ordre 0.975 de la loi normale
print("L'intervalle d'acceptation sur U à 1 − α = {:.2f} est IA = [{:.2f}, {:.2f}]".format(alpha, -u_alpha, u_alpha))

L'intervalle d'acceptation sur U à 1 − α = 0.05 est IA = [-1.96, 1.96]


In [16]:
u = (proba_traitement - proba_controle) / std
print('Valeur de u : {:.4f}'.format(u))

Valeur de u : -1.3108


In [17]:
if math.fabs(u) > u_alpha:
    print("On rejette l'hypothèse nulle")
else:
    print("On ne rejette pas l'hypothèse nulle")

On ne rejette pas l'hypothèse nulle


**Réalisons l'AB-testing en suivant la méthode proposée dans le cours**

In [18]:
d_min = 0.02 # On lance la version experimentale si on observe un changement de l'ordre de x%
beta = 0.05
alpha = 0.05
sensibilite = 1 - beta

In [19]:
effect_size = sms.proportion_effectsize(proba_controle, proba_controle + d_min)
sample_size = sms.NormalIndPower().solve_power(effect_size=effect_size,
                                               power=sensibilite, 
                                               alpha=alpha,
                                               ratio=1)
print("Taille minimum des groupes :", round(sample_size))

Taille minimum des groupes : 7357


L'objectif est de trouver un intervalle de confiance pour la différence entre les deux proportions. Pour conclure, il faut comparer d_min.

In [20]:
d_estime = (proba_traitement - proba_controle)
marge = std * u_alpha
borne_inf = d_estime - marge
borne_sup = d_estime + marge

print('Valeur estimee de d : {:.4f}'.format(d_estime))

print("L'intervalle confiance pour d à 1 − α = {:.2f} est IC = [{:.5f}, {:.5f}]".format(alpha, borne_inf, borne_sup))

Valeur estimee de d : -0.0016
L'intervalle confiance pour d à 1 − α = 0.05 est IC = [-0.00394, 0.00078]


D'après l'intervalle de confiance de d, d est inférieure à d_min, on ne lance pas la version experimentale.